In [ ]:
import os
from google.cloud import bigquery
import google.generativeai as genai

# --- Configuration ---
# IMPORTANT: Replace with your actual project and table details.
PROJECT_ID = "qwiklabs-gcp-02-e6e6123d96ed"
# The BigQuery table containing the questions, answers, and embeddings.
BIGQUERY_TABLE = "my_data_faq.aurora_bay_faq_embedded" 
# The BigQuery embedding model used to create the vectors.
BIGQUERY_EMBEDDING_MODEL = "my_data_faq.Embeddings"
# Name of the environment variable for your API key.
GEMINI_API_KEY_ENV_VAR = "GEMINI_API_KEY"
# The Gemini model to use for generating responses.
GEMINI_MODEL_NAME = "gemini-pro"


# --- Helper function for printing (works in both terminal and notebooks) ---
def is_notebook():
    """Checks if the code is running in a notebook environment."""
    try:
        from IPython import get_ipython
        if 'IPython' in get_ipython().__class__.__name__:
            return True
    except (ImportError, AttributeError):
        pass
    return False

# Use Markdown for rich output in notebooks, plain text for terminals.
_IS_NOTEBOOK = is_notebook()
if _IS_NOTEBOOK:
    from IPython.display import display, Markdown

def print_markdown(text):
    """Prints text as Markdown in notebooks, or plain text in terminals."""
    if _IS_NOTEBOOK:
        display(Markdown(text))
    else:
        # A simple conversion for terminal display
        text = text.replace('**', '') # Remove bold markers
        text = text.replace('`', '')  # Remove code block markers
        print(text)


def initialize_services():
    """Initializes BigQuery and Gemini services."""
    print("--- Initializing Services ---")
    try:
        bq_client = bigquery.Client(project=PROJECT_ID)
        print(f"✅ BigQuery client connected to project: {PROJECT_ID}")
    except Exception as e:
        print(f"❌ BigQuery connection failed. Please check project ID and authentication. Error: {e}")
        return None, None

    try:
        api_key = os.environ.get(GEMINI_API_KEY_ENV_VAR)
        if not api_key:
            raise ValueError(f"API key not found. Please set the '{GEMINI_API_KEY_ENV_VAR}' environment variable.")
        
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel(GEMINI_MODEL_NAME)
        print(f"✅ Gemini model '{GEMINI_MODEL_NAME}' is ready.")
        return bq_client, model

    except Exception as e:
        print(f"❌ Gemini setup failed: {e}")
        return bq_client, None


def search_knowledge_base(bq_client, user_question):
    """Searches the BigQuery knowledge base using VECTOR_SEARCH."""
    print(f"🔍 Searching for context related to: '{user_question}'")
    
    # This query finds the single most relevant document using vector similarity.
    search_query = f"""
    SELECT
        base.content,
        base.question
    FROM
        VECTOR_SEARCH(
            TABLE `{PROJECT_ID}.{BIGQUERY_TABLE}`,
            'ml_generate_embedding_result',
            (
                SELECT ml_generate_embedding_result
                FROM ML.GENERATE_EMBEDDING(
                    MODEL `{PROJECT_ID}.{BIGQUERY_EMBEDDING_MODEL}`,
                    (SELECT '{user_question}' AS content)
                )
            ),
            top_k => 1,
            -- This option improves performance on very large datasets.
            options => '{{"fraction_lists_to_search": 0.01}}'
        );
    """

    try:
        query_job = bq_client.query(search_query)
        results = list(query_job.result()) # Use list() to wait for completion

        if results:
            # We return the 'content' (the answer) of the most similar row.
            return results[0].content
        return None # Return None if no results are found

    except Exception as e:
        print(f"⚠️ Vector search failed: {e}")
        return None


def generate_response(model, user_question, context):
    """Generates a conversational AI response using the retrieved context."""
    print("🤖 Generating conversational response...")
    
    # This prompt instructs the AI to be helpful but stay within the provided context.
    system_prompt = f"""
You are the Aurora Bay Information Assistant. Your role is to provide friendly and helpful answers based ONLY on the information provided below.
If the answer is not in the information, you must politely state that you cannot find the answer in your knowledge base. Do not make up information.

**Provided Information:**
---
{context}
---

**User's Question:** {user_question}

**Your Answer:**
"""

    try:
        response = model.generate_content(system_prompt)
        return response.text.strip()
    except Exception as e:
        print(f"⚠️ Response generation failed: {e}")
        return "I'm sorry, I encountered an issue and can't generate a response right now."


def run_chatbot():
    """Main chatbot interaction loop."""
    bq_client, model = initialize_services()

    if not bq_client or not model:
        print("\n❌ Chatbot cannot start due to initialization failure. Please check your configuration and credentials.")
        return
    
    print_markdown("\n--- 🏔️ **Aurora Bay Information Assistant** ---")
    print_markdown("💬 Ask me anything about our products, shipping, or returns!")
    print_markdown("*(Type 'quit' or 'exit' to end the conversation)*\n")

    while True:
        try:
            user_input = input("👤 You: ").strip()

            if user_input.lower() in {"quit", "exit", "bye"}:
                print("\n👋 Assistant: Thanks for visiting Aurora Bay! Have a great day! 🌟")
                break

            if not user_input:
                continue

            # Step 1: Search BigQuery for the most relevant information
            context = search_knowledge_base(bq_client, user_input)

            if not context:
                print("\n🤖 Assistant: I'm sorry, I couldn't find any information related to that topic. Could you please try asking in a different way?\n")
                continue

            # Step 2: Use Gemini to generate a natural response from the found context
            answer = generate_response(model, user_input, context)
            
            # Step 3: Display the final answer
            print_markdown(f"\n🤖 **Assistant:** {answer}\n")

        except KeyboardInterrupt:
            print("\n\n👋 Assistant: Conversation ended. Goodbye!")
            break
        except Exception as e:
            print(f"\n⚠️ An unexpected error occurred: {e}\n")


if __name__ == "__main__":
    run_chatbot()


Loading embedding model 'all-MiniLM-L6-v2'...
Generating embeddings for the FAQ data...
Embeddings are ready.

--- Chatbot is now active! ---
Ask me a question. Type 'quit' or 'exit' to end the chat.
------------------------------
You: What are your shipping options?

Bot: It looks like you're asking about 'What are your shipping options?'.
Here is the information I found: We offer standard (5-7 business days), expedited (2-3 business days), and overnight shipping.

You: quit

Bot: Goodbye! Have a great day.
